# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
data_df = pd.read_csv("city_weather_data.csv")
data_df = data_df.drop(['Unnamed: 0'], axis=1)
data_df

,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
0,hermanus,-34.4187,19.2345,287.48,73,100,4.66,ZA,1629136234
1,dingle,10.9995,122.6711,298.04,90,97,2.45,PH,1629136234
2,ayolas,-27.4000,-56.9000,300.98,38,5,2.16,PY,1629136235
3,cheney,47.4874,-117.5758,304.12,24,20,9.26,US,1629136235
4,mount gambier,-37.8333,140.7667,282.10,93,90,3.60,AU,1629136236
...,...,...,...,...,...,...,...,...,...
543,ensley,30.5188,-87.2728,301.81,81,75,1.79,US,1629136433
544,axim,4.8699,-2.2405,298.63,74,98,3.96,GH,1629136434
545,namatanai,-3.6667,152.4333,298.55,82,100,1.17,PG,1629136434
546,mbanza-ngungu,-5.2500,14.8667,300.07,53,91,3.09,CD,1629136434


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#g maps set up
gmaps.configure(api_key=g_key)

#set up heat map locations
locations = data_df[["lat", "lng"]]

#set up humidities
humidity = data_df["humidity"]

#create heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
filtered_df = data_df.loc[(data_df["wind speed"] < 10) \
                          & (data_df["cloudiness"] == 0) \
                          & (data_df["max temp"] > 294.261) \
                          & (data_df["max temp"] < 299.817)]

filtered_df

,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,date
44,peniche,39.3558,-9.3811,298.86,81,0,9.54,PT,1629136249
76,rikitea,-23.1203,-134.9692,296.91,76,0,6.02,PF,1629136261
93,nurota,40.5614,65.6886,297.20,28,0,7.82,UZ,1629136267
94,disraeli,45.9001,-71.3491,297.71,44,0,1.79,CA,1629136268
218,arraial do cabo,-22.9661,-42.0278,298.13,73,0,8.23,BR,1629136314
219,muros,42.7762,-9.0603,298.84,66,0,7.52,ES,1629136314
270,cerkezkoy,41.2850,28.0003,297.16,68,0,5.66,TR,1629136332
278,ondjiva,-17.0667,15.7333,295.81,19,0,1.58,AO,1629136335
295,xinpu,34.5997,119.1594,295.12,92,0,0.49,CN,1629136342
402,sabzevar,36.2126,57.6819,298.33,13,0,6.64,IR,1629136141


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = filtered_df
hotel_df["hotel name"] = ""
hotel_df.head()

<ipython-input-15-5d39b89d40a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["hotel name"] = ""


,city,lat,lng,max temp,humidity,cloudiness,wind speed,country,date,hotel name
44,peniche,39.3558,-9.3811,298.86,81,0,9.54,PT,1629136249,
76,rikitea,-23.1203,-134.9692,296.91,76,0,6.02,PF,1629136261,
93,nurota,40.5614,65.6886,297.20,28,0,7.82,UZ,1629136267,
94,disraeli,45.9001,-71.3491,297.71,44,0,1.79,CA,1629136268,
218,arraial do cabo,-22.9661,-42.0278,298.13,73,0,8.23,BR,1629136314,


In [ ]:
#set params for search
params = {"radius" : 5000, "types" : "lodging", "key" : g_key, "location" : ""}

#set base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#for loop set up
for city in hotel_df:
    
    #get locations
    lat = city["lat"]
    lng = city["lng"]
    
    #add to params
    params["location"] = f"{lat},{lng}"
    
    #request
    response = requests.get(base_url, params=params)
    
    #make it a json
    response = response.json()
    
    #find first hotel name and add to df
    
    


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
